In [1]:
import h2o4gpu
import h2o4gpu.util.import_data as io
import h2o4gpu.util.metrics as metrics
from tabulate import tabulate
import pandas as pd

In [2]:
"""
Import Data for H2O GPU Edition

This function will read in data and prepare it for H2O4GPU's GLM solver

Parameters
----------
data_path : str
             A path to a dataset (The dataset needs to be all numeric)
use_pandas : bool
              Indicate if Pandas should be used to parse
intercept : bool
              Indicate if intercept term is needed
valid_fraction : float
                  Percentage of dataset reserved for a validation set
classification : bool
                  Classification problem?
Returns
-------
If valid_fraction > 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    valid_x: numpy array of valid input variables
    valid_y: numpy array of valid y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
If valid_fraction == 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
"""

train_x,train_y,valid_x,valid_y,family=io.import_data(data_path="../../../data/credit.feather", 
                                                        use_pandas=False, 
                                                        intercept=True,
                                                        valid_fraction=0.2,
                                                        classification=True)

Reading Data with Feather
(23999, 25)
Original m=23999 n=24
Size of Train rows=19200 & valid rows=4799
Size of Train cols=24 valid cols=24
Size of Train cols=25 & valid cols=25 after adding intercept column


In [3]:
"""
Set up instance of H2O4GPU's Lasso solver with default parameters

Need to pass in `family` to indicate problem type to solve
"""
print("Setting up solver")
model = h2o4gpu.Lasso(family=family)

Setting up solver


In [4]:
"""
Fit Lasso Solver
"""
print("Solving")
%time model.fit(train_x, train_y)
print("Done Solving")
model.summary()

Solving
CPU times: user 880 ms, sys: 952 ms, total: 1.83 s
Wall time: 1.88 s
Done Solving
Logloss per alpha value (-1.00 = missing)

|   Alphas |   Train |    CV |   Valid |
|---------:|--------:|------:|--------:|
|     1.00 |    0.48 | -1.00 |   -1.00 |


In [5]:
"""
Make predictions on validation set
"""
print("Predictions per alpha")
preds = model.predict(valid_x, valid_y)
print(preds)

Predictions per alpha
[[ 0.77694041  0.14217262  0.14151368 ...,  0.2666837   0.24216804
   0.16727191]]


In [6]:
"""
Get logloss or rmse for validation set per alpha
"""
for i in range(model.n_alphas):
    if family == "logistic":
        print("Logloss for alpha = ",model.alphas_best[i])
        print(metrics.log_loss(valid_y, preds[i]))
    else:
        print("RMSE for alpha = ",model.alphas_best[i])
        print(metrics.rmse(valid_y,preds[i]))

Logloss for alpha =  [ 1.]
0.438997779266
